### This notebook is the continuation of the featurization_notebook where we got the features and saved them to a file.

In [1]:
import sys
sys.path.append('../modules')
import likelihood_predictor
from likelihood_predictor import PlastPredictor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import zscore
import pickle
from sklearn.feature_selection import SelectFromModel

In [49]:
pl_full = pd.read_pickle('../database/plasticizer_data_v10_polarity.pkl')
pl_pol = pd.concat([pl_full[pl_full.columns[1:195]], pl_full['Polarity']], axis=1)
all_cols = pl_pol.columns.to_numpy()
pl_data = pl_pol[all_cols].to_numpy()
lin_data = pd.read_pickle('../database/linolein_test.pkl')
lin_data['Polarity'] = 0.048856
lin_data = lin_data[all_cols].to_numpy()
org_full = pd.read_pickle('../database/org_polarity_v2.pkl')

In [50]:
psim1 = open("pubs_similarity.txt", 'r')
psim11 = [line.rstrip('\n') for line in psim1]
psim2 = open("pubs_othersim.txt", 'r')
psim22 = [line.rstrip('\n') for line in psim2]

In [51]:
org_full
org_full['Dsim'] = psim11
org_full['Nasim'] = psim22
org_full = org_full.sort_values(by ='Dsim')

In [58]:
org_full = org_full[:5000]
org_data = org_full[all_cols]

In [24]:
pl_full = pd.read_excel('feature.xls')

This is our df. Ignore the last column. We are focusing on the **zero** column, which are the positions of our relevant features.

In [26]:
pl_full.columns = ['zero', 'pl_l', 'pl_list38', 'org_list37', 'zz']

In [44]:
pl_full = pl_full.sort_values(by='zero', ascending=True)

In [45]:
pl_list = pl_full['pl_l'].to_list()

In [46]:
pl_full

,zero,pl_l,pl_list38,org_list37,zz
16,0,11.863075,11.863075,8.659722,6.760439
0,1,-0.044431,-0.044431,-1.611111,7.277336
1,5,0.000000,0.000000,0.000000,-31.974731
15,11,3.555747,3.555747,3.023716,-2.475761
2,33,0.000000,0.000000,8.317393,-9.307438
3,40,0.000000,0.000000,0.000000,3.172372
17,46,11.938611,11.938611,8.317393,-0.857588
18,51,13.213764,13.213764,0.000000,-12.177630
20,74,25.683286,25.683286,0.000000,-18.588882
4,82,0.000000,0.000000,0.000000,-3.250218


In [47]:
positions = pl_full['zero'].to_list()

In [48]:
positions

[0,
 1,
 5,
 11,
 33,
 40,
 46,
 51,
 74,
 82,
 93,
 102,
 106,
 111,
 117,
 132,
 149,
 150,
 177,
 183,
 185]

In [67]:
feat = pl_pol.columns.to_list()

In [68]:
sel_ft = []
for i in positions:
    sel_ft.append(feat[i])

In [71]:
sel_ft

['MaxEStateIndex',
 'MinEStateIndex',
 'NumRadicalElectrons',
 'BalabanJ',
 'PEOE_VSA11',
 'PEOE_VSA5',
 'SMR_VSA10',
 'SMR_VSA6',
 'EState_VSA5',
 'VSA_EState3',
 'NumAliphaticCarbocycles',
 'NumRotatableBonds',
 'RingCount',
 'fr_Al_OH_noTert',
 'fr_COO',
 'fr_aldehyde',
 'fr_ester',
 'fr_ether',
 'fr_phos_ester',
 'fr_quatN',
 'fr_sulfonamd']

In [80]:
pl_pol=pl_pol.T

In [82]:
pl_new = pl_pol.loc[sel_ft].T

In [83]:
pl_new

,MaxEStateIndex,MinEStateIndex,NumRadicalElectrons,BalabanJ,PEOE_VSA11,PEOE_VSA5,SMR_VSA10,SMR_VSA6,EState_VSA5,VSA_EState3,...,NumRotatableBonds,RingCount,fr_Al_OH_noTert,fr_COO,fr_aldehyde,fr_ester,fr_ether,fr_phos_ester,fr_quatN,fr_sulfonamd
0,11.863075,-0.044431,0.0,3.555747e+00,0.000000,0.00000,11.938611,13.213764,25.683286,0.000000,...,21.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
1,12.421299,-0.445911,0.0,2.322002e+00,0.000000,0.00000,11.938611,13.213764,25.683286,0.000000,...,16.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
2,11.276404,-4.535630,0.0,1.706666e-06,0.000000,0.00000,41.781151,0.000000,24.265468,2.907510,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.944745,-0.470841,0.0,2.712391e+00,0.000000,0.00000,11.938611,13.213764,0.000000,0.000000,...,8.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
4,11.268799,-0.181252,0.0,3.435893e+00,0.000000,0.00000,11.938611,13.213764,0.000000,0.000000,...,9.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,12.960039,-0.610511,0.0,2.724699e+00,0.000000,0.00000,17.907916,19.820646,75.986496,0.000000,...,24.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0
206,5.117870,0.956019,0.0,1.111111e-07,0.000000,0.00000,17.045264,0.000000,11.126903,1.164352,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207,11.861333,-0.950556,0.0,-7.620866e-07,12.207933,0.00000,59.693053,52.855056,388.266791,19.009168,...,86.0,0.0,2.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0
208,12.889740,-0.120563,0.0,1.159613e+00,0.000000,23.20188,40.765751,29.749429,19.262465,3.371101,...,8.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [91]:
org_data = org_data.T

In [94]:
org_new = org_data.loc[sel_ft].T

In [96]:
org_new.to_excel('new_selected_org.xls')

In [97]:
pl_new.to_excel('new_selected_pl.xls')

In [98]:
org_num = org_new.to_numpy()

In [99]:
pl_num = pl_new.to_numpy()

In [100]:
reg_param = 10
pp = PlastPredictor(reg_param)
pp_model = pp.fit_model(pl_num, org_num)


In [103]:
org_acc = pp.predict(org_num, type='binary', class_id='neg')
pl_acc = pp.predict(pl_num, type='binary', class_id='pos')
cc=pp.clf.coef_

In [104]:
np.count_nonzero(cc), org_acc, pl_acc

(21, 0.9554, 0.919047619047619)

In [107]:
sel_ft

['MaxEStateIndex',
 'MinEStateIndex',
 'NumRadicalElectrons',
 'BalabanJ',
 'PEOE_VSA11',
 'PEOE_VSA5',
 'SMR_VSA10',
 'SMR_VSA6',
 'EState_VSA5',
 'VSA_EState3',
 'NumAliphaticCarbocycles',
 'NumRotatableBonds',
 'RingCount',
 'fr_Al_OH_noTert',
 'fr_COO',
 'fr_aldehyde',
 'fr_ester',
 'fr_ether',
 'fr_phos_ester',
 'fr_quatN',
 'fr_sulfonamd']